In [5]:
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore

In [6]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

In [7]:
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    # deployment_name="my-custom-embedding",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("OPENAI_API_VERSION"),
)
embeddings = embed_model.get_text_embedding("Hello, world!")
len(embeddings)

1536

In [3]:
neo4j_vector = Neo4jVectorStore(
    username="neo4j",
    password="12345678",
    url="bolt://localhost:7687",
    database="neo4j",
    hybrid_search = True,
    embedding_dimension=1536
)

In [17]:
neo4j_vector.retrieve_existing_index()

True

In [10]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
storage_context = StorageContext.from_defaults(vector_store=neo4j_vector)
index = VectorStoreIndex.from_vector_store(neo4j_vector, embed_model=embed_model)


In [15]:
retriever = index.as_retriever(similarity_top_k=5)

In [16]:
retriever.retrieve("Tell me about Y Combinator")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: 'CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN node.`text` AS text, score, node.id AS id, node {.*, `text`: Null, `embedding`: Null, id: Null } AS metadata

[NodeWithScore(node=TextNode(id_='830ca6e3-e6ca-4682-999b-e08d09efbc33', embedding=None, metadata={'file_path': "D:\\projects\\graphrag-poc\\data\\selection 1\\A Student's Guide to Startups (HIVE).txt", 'file_name': "A Student's Guide to Startups (HIVE).txt", 'file_type': 'text/plain', 'file_size': 35856, 'creation_date': '2024-11-26', 'last_modified_date': '2024-11-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='afc37d27-1b7a-4bb4-af36-4aeb9c5036b3', node_type='4', metadata={'file_path': "D:\\projects\\graphrag-poc\\data\\selection 1\\A Student's Guide to Startups (HIVE).txt", 'file_name': "A Student's Guide to Startups (HIVE).txt", 'file_type': 'text/plain', 'file_size': 35856, 'creation_date': '202

In [1]:
from neo4j import GraphDatabase

# Demo database credentials
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")
# Connect to Neo4j database
driver = GraphDatabase.driver(URI, auth=AUTH, database="graphrag-openai-custom-extractor-prompt-gemini")


In [ ]:
from neo4j_graphrag.retrievers import HybridCypherRetriever